# SpaCy Matcher test

In [13]:
import pandas as pd

# path = 'C:/Users/Gebruiker/Documents/UA/Stage/Middelner_Github/C14NL-PoS-annotated/TEST.tsv'
path = '../C14NL-PoS-annotated/TEST.tsv'
TEST_DF = pd.read_csv(path, sep="\t", encoding = "UTF-8", header=None)


from string import punctuation

def clean_words(word):
  word = word.casefold()
  word = word.translate(str.maketrans('', '', punctuation))
  return word

# print(TRAIN[0])
TEST_DF[0] = TEST_DF[0].apply(clean_words)

TEST_DF.head()

,0,1,2,3,4
0,wie,O,688,H072p40001,140
1,suster,O,688,H072p40001,140
2,lisebette,B-PERS,688,H072p40001,140
3,bi,O,688,H072p40001,140
4,den,O,688,H072p40001,140


In [41]:
body = []

grouped = TEST_DF.groupby(3)

for group in grouped.groups:
    df = grouped.get_group(group)
    body.append(' '.join(list(df[0])))


grouped = TEST_DF.groupby(3)

tags = []

for group in grouped.groups:
    df = grouped.get_group(group)
    df[1] = ['NAME' if x == 'B-PERS' or x =='I-PERS' else 'O' for x in df[1] ]
    tags.append(list(df[1]))

X = body
y = tags

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_16648\1836481126.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1] = ['NAME' if x == 'B-PERS' or x =='I-PERS' else 'O' for x in df[1] ]


In [48]:
import spacy
import re
from spacy.language import Language
from spacy.matcher import Matcher
from spacy.tokens import Span

firstnames = pd.read_csv('christian_name.txt', sep='\t', header=0, encoding='UTF-16')

lastnames = pd.read_csv('family_name.txt', sep='\t', header=0, encoding='UTF-16')

lastnames = list(lastnames["family_name_document"])
lastnames_imp = []
for lastname in lastnames:
    index = lastname.find(',')
    if index != -1:
        assert type(index) == int
        lastname = lastname[:index] + lastname[index+1:]
        lastnames_imp.append(lastname)
    else:
        lastnames_imp.append(lastname)

#load Dutch
nlp = spacy.load("nl_core_news_sm")

names = list(firstnames["christian_name_document"]) + lastnames_imp
names = set(names)
name_patterns = [[{"LOWER" : name.casefold()}] for name in names]

print(name_patterns[:20])
assert type(name_patterns) == list
matcher = Matcher(nlp.vocab)
matcher.add("NAME", name_patterns)

# Define the custom component
@Language.component("personal_name_component")
def firstname_comp_function(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label "PERSON_NAME"
    spans = [Span(doc, start, end, label="NAME") for match_id, start, end in matches]
    doc.ents = spans
    return doc

# Add the component to the pipeline after the "ner" component
nlp.add_pipe("personal_name_component", after="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp.pipe(X)
y_pred = [token.ent_type_ if len(token.ent_type_) != 0 else 'O' for Doc in doc for token in Doc]

C:\Users\Gebruiker\anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'nl_core_news_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[[{'LOWER': 'le loire'}], [{'LOWER': 'le duc'}], [{'LOWER': 'conins le machecliers'}], [{'LOWER': 'herluinus'}], [{'LOWER': 'adroaldus'}], [{'LOWER': 'everlin'}], [{'LOWER': 'de nuefport'}], [{'LOWER': 'eustachius'}], [{'LOWER': 'sprincscove'}], [{'LOWER': 'boileuwe'}], [{'LOWER': 'de vlargslo'}], [{'LOWER': 'iuliotus'}], [{'LOWER': 'foursin'}], [{'LOWER': 'dral'}], [{'LOWER': 'le pissonniers'}], [{'LOWER': 'sigitrudis'}], [{'LOWER': 'de strasieles'}], [{'LOWER': 'de bruges fil will de bruges'}], [{'LOWER': 'ermenfredus'}], [{'LOWER': 'le vattre'}]]
['tok2vec', 'morphologizer', 'tagger', 'parser', 'ner', 'personal_name_component', 'attribute_ruler', 'lemmatizer']


In [51]:
y_true = []

for doc in y:
    y_true.extend(doc)


from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

        NAME       0.71      0.19      0.29       478
           O       0.92      0.99      0.96      4772

    accuracy                           0.92      5250
   macro avg       0.82      0.59      0.63      5250
weighted avg       0.90      0.92      0.90      5250

